In [1]:
import pandas as pd
from utils import transformers_bert_completions, load_splits, load_models
from utils_model_sampling import sample_across_models, hyperparameter_utils

from yyy_analysis import examples_figure
import numpy as np

import os
from os.path import join, exists

import childespy
%load_ext autotime
%load_ext line_profiler
%pdb

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

Automatic pdb calling has been turned ON
time: 3.44 ms (started: 2022-01-31 07:53:39 -08:00)


In [2]:
import configuration
config = configuration.Config()

time: 1.85 ms (started: 2022-01-31 07:53:39 -08:00)


In [3]:
#os.environ["CUDA_VISIBLE_DEVICES"]="" #hangs as of 1/26 if GPU is enabled on workstation

time: 7.54 ms (started: 2022-01-31 07:53:39 -08:00)


# Success Example

In [4]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



time: 1.95 s (started: 2022-01-31 07:53:39 -08:00)


## Load phonology and context

In [5]:
all_tokens_phono = load_splits.load_phono()

time: 7.89 s (started: 2022-01-31 07:53:41 -08:00)


In [6]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


time: 48.3 ms (started: 2022-01-31 07:53:49 -08:00)


In [7]:
target_transcript_id = 42336 # Corresponds to the success_idx 

time: 643 µs (started: 2022-01-31 07:53:49 -08:00)


In [8]:
# Broader context of sentences
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


time: 37.6 ms (started: 2022-01-31 07:53:49 -08:00)


## Estimate Prior and Posterior Probability

In [80]:
# Changed this notebook to use the +/- 20 context newly generated versions.

# Written to match load_models.query_model_title
child_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : True,
    'context_num' : 20,
}

adult_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : False,
    'context_num' : 20,
}

childes_all_title = load_models.query_model_title(model_type = 'childes', **child_args)
adult_all_title = load_models.query_model_title(model_type = 'adult', **adult_args)
unigram_title = 'CHILDES Unigram'
flat_title = 'Flat Unigram'

time: 3.07 ms (started: 2022-01-31 08:32:35 -08:00)


In [81]:
childes_all_title

'CHILDES BERT with tags, , +-20 utts context'

time: 3.31 ms (started: 2022-01-31 08:32:37 -08:00)


In [82]:
from utils import wfst
from utils_model_sampling import hyperparameter_utils

time: 903 µs (started: 2022-01-31 08:32:38 -08:00)


In [157]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)
imp.reload(wfst)
imp.reload(hyperparameter_utils)

/home/stephan/notebooks/nicole/child-directed-listening/yyy_analysis/examples_figure.py:8: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py:6: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py:14: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/hyperparameter_utils.py:7: ResourceWarning: un

<module 'utils_model_sampling.hyperparameter_utils' from '/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/hyperparameter_utils.py'>

time: 17.6 ms (started: 2022-01-31 21:42:52 -08:00)


In [158]:
# CDL + Context +/- 20 is needed
# BERT + Context +/- 20 is needed
# Childes on train data.

# How to load properly with sample across models?
which_models = [
    ('all', 'all', False, 0, 'data_unigram'),
    ('all', 'all', True, 20, 'childes'), # only the fine-tuned one can have tags
    ('all', 'all', False, 20, 'adult'),
    ('all', 'all', False, 0, 'flat_unigram'),
]

time: 1.65 ms (started: 2022-01-31 21:42:56 -08:00)


In [ ]:
raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, which_models, True)

Running model CHILDES Unigram...
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

> /home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py(61)sample_across_models()
     59     pdb.set_trace()
     60 
---> 61     for idx, lambda_value in enumerate(lambda_values):
     62 
     63         print(f'Processing lambda value {idx + 1} of {config.lambda_num_values}')



ipdb>  wfst_distances_for_age_interval.shape


(4, 7997)


ipdb>  wfst_distances_for_age_interval_unreduced.shape


(4, 8943)


In [150]:
scores_across_models = pd.concat(raw_scores_across_models)

time: 14.9 ms (started: 2022-01-31 15:09:53 -08:00)


In [151]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'wfst']

time: 19.5 ms (started: 2022-01-31 15:09:53 -08:00)


In [152]:
np.unique(scores_across_models.likelihood_type)

array(['wfst'], dtype=object)

time: 4.53 ms (started: 2022-01-31 15:09:53 -08:00)


In [153]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','posterior_probability','token','likelihood_type','prior_rank','posterior_rank']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,posterior_probability,token,likelihood_type,prior_rank,posterior_rank
997721,"CHILDES BERT with tags, , +-20 utts context",read wait weed what with watch lead reach read...,0.994283890865658 0.0009247716770197329 0.0006...,see go play read look do watch draw know eat,0.5455823 0.0501516 0.042885385 0.034582537 0....,0.034583,0.994284,read,wfst,3,0.0


time: 29.8 ms (started: 2022-01-31 15:09:53 -08:00)


In [154]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
print('CDL+Context Prior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Prior


'see (0.55) go (0.05) play (0.04) read (0.03) look (0.02) do (0.02) watch (0.02) draw (0.01) know (0.01) eat (0.01)'

time: 14.9 ms (started: 2022-01-31 15:09:53 -08:00)


In [155]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('CDL+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],3))+')' for i in range(len(words))]) 

CDL+Context Posterior


'read (0.994) wait (0.001) weed (0.001) what (0.001) with (0.001) watch (0.0) lead (0.0) reach (0.0) reading (0.0) wonder (0.0)'

time: 19.7 ms (started: 2022-01-31 15:09:53 -08:00)


In [135]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

print('BERT+Context Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

BERT+Context Prior


'read (0.49) see (0.28) play (0.04) know (0.04) look (0.02) go (0.01) watch (0.01) help (0.01) try (0.01) talk (0.01)'

time: 13.7 ms (started: 2022-01-31 08:54:30 -08:00)


In [136]:
adult_all_title

'Adult BERT without tags, , +-20 utts context'

time: 15.6 ms (started: 2022-01-31 08:54:30 -08:00)


In [137]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('BERT+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],4))+')' for i in range(len(words))]) 

BERT+Context Posterior


'read (0.9997) wait (0.0001) weed (0.0001) lead (0.0) reads (0.0) reading (0.0) wonder (0.0) watch (0.0) write (0.0) reader (0.0)'

time: 18.1 ms (started: 2022-01-31 08:54:30 -08:00)


In [156]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token', 'prior_rank', 'posterior_rank']]
#print(success_example)
print('CHILDES-1Gram Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1Gram Prior


'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 27.1 ms (started: 2022-01-31 15:15:53 -08:00)


In [139]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('CHILDES-1gram Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1gram Posterior


'read (0.46) with (0.24) what (0.1) would (0.04) we (0.03) weed (0.02) word (0.01) one (0.01) wood (0.01) wait (0.01)'

time: 15.4 ms (started: 2022-01-31 08:55:20 -08:00)


In [140]:
success_example = scores_across_models.loc[(scores_across_models.model == flat_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example
print('UniformPrior Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Prior


'stalks (0.0) rarely (0.0) harbor (0.0) dust (0.0) josh (0.0) milk (0.0) leon (0.0) pitch (0.0) blake (0.0) girlfriend (0.0)'

time: 22.3 ms (started: 2022-01-31 08:55:22 -08:00)


In [141]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('UniformPrior Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Posterior


'weed (0.37) worried (0.17) reed (0.06) read (0.06) wheeled (0.05) wheat (0.03) weeds (0.03) reader (0.03) hurried (0.03) lead (0.02)'

time: 18.3 ms (started: 2022-01-31 08:55:23 -08:00)


# Failure Example

In [142]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)

/home/stephan/notebooks/nicole/child-directed-listening/yyy_analysis/examples_figure.py:8: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py:6: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py:14: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()


<module 'utils.load_models' from '/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py'>

time: 11 ms (started: 2022-01-31 08:55:34 -08:00)


In [143]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, which_models, False)
scores_across_models = pd.concat(raw_scores_across_models)

Running model CHILDES Unigram...
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Running model CHILDES BERT with tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20


...

Running model Adult BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
Running model Flat Unigram...
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/likelihoods.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
time: 42.2 s (started: 2022-01-31 08:55:36 -08:00)


In [101]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
289678,you make your yyy,,,16813515,289678,112,42253
289679,you make your yyy,ju,ju,16813515,289679,112,42253
289680,you make your yyy,meək,meək,16813515,289680,112,42253
289681,you make your yyy,jɜ,jɑɹ,16813515,289681,112,42253
289682,you make your yyy,fɜt,*,16813515,289682,112,42253
289683,you make your yyy,,,16813515,289683,112,42253


time: 44.1 ms (started: 2022-01-31 08:34:48 -08:00)


In [102]:
scores_across_models

,token,bert_token_id,prior_entropy,set,keep,posterior_entropy,posterior_rank,position_in_mask,kl_flat_to_prior,kl_flat_to_posterior,...,highest_prior_probabilities,sample_index,lambda_value,model,likelihood_type,beta_value,prior_rank,prior_prob,entropy,num_tokens_in_context
289682,yyy,289682.0,5.837648,failure,True,2.040508,NaN,NaN,NaN,NaN,...,0.04249687198055286 0.032549581544440515 0.027...,0,1.1,CHILDES Unigram,wfst,NaN,NaN,NaN,NaN,NaN
289682,yyy,289682.0,5.837648,failure,True,4.163917,NaN,NaN,NaN,NaN,...,0.04249687198055286 0.032549581544440515 0.027...,0,1.1,CHILDES Unigram,levdist,3.5,NaN,NaN,NaN,NaN
289682,NaN,289682.0,NaN,failure,NaN,2.270785,NaN,NaN,NaN,NaN,...,0.16272134 0.046508007 0.04586447 0.029345786 ...,0,1.1,"CHILDES BERT with tags, , +-20 utts context",wfst,NaN,NaN,NaN,7.817930,16.0
289682,NaN,289682.0,NaN,failure,NaN,3.559607,NaN,NaN,NaN,NaN,...,0.16272134 0.046508007 0.04586447 0.029345786 ...,0,1.1,"CHILDES BERT with tags, , +-20 utts context",levdist,3.1,NaN,NaN,7.817930,16.0
289682,NaN,289682.0,NaN,failure,NaN,2.188450,NaN,NaN,NaN,NaN,...,0.2485145 0.24318402 0.04469067 0.033207614 0....,0,1.1,"Adult BERT without tags, , +-20 utts context",wfst,NaN,NaN,NaN,5.123916,13.0
289682,NaN,289682.0,NaN,failure,NaN,2.794463,NaN,NaN,NaN,NaN,...,0.2485145 0.24318402 0.04469067 0.033207614 0....,0,1.1,"Adult BERT without tags, , +-20 utts context",levdist,3.5,NaN,NaN,5.123916,13.0
289682,yyy,289682.0,8.986822,failure,True,3.521773,NaN,NaN,NaN,NaN,...,0.00012504689258471927 0.00012504689258471927 ...,0,1.3,Flat Unigram,wfst,NaN,NaN,NaN,NaN,NaN
289682,yyy,289682.0,8.986822,failure,True,4.516821,NaN,NaN,NaN,NaN,...,0.00012504689258471927 0.00012504689258471927 ...,0,1.3,Flat Unigram,levdist,3.7,NaN,NaN,NaN,NaN


time: 58.3 ms (started: 2022-01-31 08:34:53 -08:00)


In [103]:
childes_all_title

'CHILDES BERT with tags, , +-20 utts context'

time: 3.28 ms (started: 2022-01-31 08:34:57 -08:00)


In [115]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','posterior_rank']]
yyy_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type,posterior_rank
289682,"CHILDES BERT with tags, , +-20 utts context",favorite first fort face feet foot front fist ...,0.34961168476313437 0.208275624182266 0.118228...,own house bed dinner face mouth head tea nose ...,0.16272134 0.046508007 0.04586447 0.029345786 ...,NaN,NaN,wfst,NaN


time: 27.8 ms (started: 2022-01-31 08:36:38 -08:00)


In [116]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

'own (0.16) house (0.05) bed (0.05) dinner (0.03) face (0.02) mouth (0.02) head (0.01) tea (0.01) nose (0.01) room (0.01)'

time: 13.4 ms (started: 2022-01-31 08:36:39 -08:00)


In [117]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'favorite (0.35) first (0.21) fort (0.12) face (0.07) feet (0.05) foot (0.05) front (0.05) fist (0.01) fruit (0.01) fit (0.01)'

time: 16.9 ms (started: 2022-01-31 08:36:40 -08:00)


In [118]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.25) choice (0.24) point (0.04) bed (0.03) call (0.03) guess (0.03) wish (0.02) choices (0.02) move (0.02) mistake (0.02)'

time: 19.1 ms (started: 2022-01-31 08:36:44 -08:00)


In [119]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'favorite (0.34) first (0.26) effort (0.14) fort (0.06) offer (0.03) threat (0.02) fist (0.01) fate (0.01) fit (0.01) freighter (0.01)'

time: 16.8 ms (started: 2022-01-31 08:36:47 -08:00)


In [122]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'a (0.04) b (0.03) c (0.03) d (0.03) e (0.03) f (0.03) g (0.02) h (0.02) i (0.02) j (0.01)'

time: 23.1 ms (started: 2022-01-31 08:37:42 -08:00)


In [121]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'first (0.38) for (0.32) front (0.08) different (0.03) fort (0.03) from (0.02) after (0.02) feet (0.01) foot (0.01) effort (0.01)'

time: 14.5 ms (started: 2022-01-31 08:37:28 -08:00)


In [123]:
yyy_example = scores_across_models.loc[(scores_across_models.model == flat_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'stalks (0.0) rarely (0.0) harbor (0.0) dust (0.0) josh (0.0) milk (0.0) leon (0.0) pitch (0.0) blake (0.0) girlfriend (0.0)'

time: 19.3 ms (started: 2022-01-31 08:38:24 -08:00)


In [124]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'freight (0.2) freighter (0.09) fort (0.09) flirt (0.08) front (0.04) fit (0.04) effort (0.03) fate (0.03) favorite (0.02) foot (0.02)'

time: 20.9 ms (started: 2022-01-31 08:38:26 -08:00)


In [125]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
50658,then we won't be able to put them back into th...,42253,16813459,109,Alex,MOT,declarative
50679,do you want ta put some beans in your eggs and...,42253,16813482,110,Alex,MOT,question
50697,no,42253,16813501,111,Alex,CHI,declarative
50710,you make your yyy,42253,16813515,112,Alex,CHI,declarative
50723,can I make one,42253,16813529,113,Alex,MOT,question
50745,no,42253,16813554,114,Alex,MOT,declarative


time: 41.4 ms (started: 2022-01-31 08:38:37 -08:00)
